In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from definitions import ROOT_DIR
import seaborn as sns

In [3]:
def calculate_detected_intensities(df, threshold=0.8):
    '''
    Make a column with background corrected intensities for detected compounds, and 0s for not detected compounds
    Change any negative values to zero
    Also add detectability column, where compounds with prediction value above threshold=0.8 are labelled as detected (1)
    '''

    df['detectability'] = df.pred_val >= threshold
    vals = df.spot_intensity_bgr_corrected * df.detectability
    df['effective_intensity'] = np.clip(vals, 0, None)
    return df

In [4]:
p_root_dir = Path(ROOT_DIR).parents[0]
p_data = p_root_dir / "5_data_analysis"

# Metrics and Catboost predictions for all ions in their target wells
p_predictions = p_data / "all_predictions_curated_11-Dec-2021.csv"

# Dataset info (lab, matrix, polarity, m/z range, ids, etc.)
p_datasets = p_data / "Datasets_01June2022.csv"

# Classification
p_chem_class = p_data / "custom_classification_v2.csv"
p_pathways = p_data / "pathways_v2.csv"

In [5]:
# Load predictions, format neutral loss column
predictions = pd.read_csv(p_predictions, index_col=0)[['dataset_id', 
                                                       'name_short', 
                                                       'adduct', 
                                                       'neutral_loss', 
                                                       'pred_val',
                                                       'spot_intensity_bgr_corrected']]
predictions.neutral_loss.fillna('', inplace=True)

In [6]:
# Add metadata
classes1 = pd.read_csv(p_chem_class, index_col='internal_id')
chem_class = classes1[['name_short', 'main_coarse_class']].drop_duplicates()
chem_subclass = classes1[['name_short', 'coarse_class', 'fine_class']] # coarse class here is to do sorting if you want

classes2 = pd.read_csv(p_pathways, index_col='internal_id')
pathways = classes2[['name_short', 'main_coarse_path']].drop_duplicates()
pathway_subclass = classes2[['name_short', 'coarse_path', 'fine_path']] # same here

# Get a subset of most relevant information from Datasets file
datasets = pd.read_csv(p_datasets)
datasets_info = datasets.groupby('Dataset ID').first()[['Polarity', 'Matrix short', 'Matrix long', 'Slide code', 'EMBL']]
datasets_info['sample_name'] = datasets_info['Matrix short']

# Merge with predictions
df = pd.merge(predictions, datasets_info, left_on='dataset_id', right_on='Dataset ID', how='left')

In [7]:
# Filter to keep only datasets chosen for plots about matrix comparison
df = df[df.EMBL]

In [8]:
# only consider data of detected ions
df = calculate_detected_intensities(df, threshold=0.8)
data = df[df.detectability]

### Here you would also have relevant filtering steps

## Example for any axis values except class, subclass, pathway and pathway subclass

In [12]:
# Aggregate data from individual ions per metabolite ('name_short'), per dataset ('dataset_id') and axis values

# for example for these axes
X='adduct'
Y='Matrix short'

step1 = data.pivot_table(index=['dataset_id', 'name_short', X, Y],
            values=['effective_intensity', 'detectability'],
            aggfunc=
                {'effective_intensity':sum,
                'detectability':max})

step1.head()

detectability  \
dataset_id           name_short                adduct Matrix short                  
2021-06-18_10h37m54s 2-Oxoglutaric acid        +Cl    9AA                    True   
                                               -H     9AA                    True   
                     3-Hydroxyanthranilic acid +Cl    9AA                    True   
                                               -H     9AA                    True   
                                               [M]-   9AA                    True   

                                                                    effective_intensity  
dataset_id           name_short                adduct Matrix short                       
2021-06-18_10h37m54s 2-Oxoglutaric acid        +Cl    9AA                    151.037020  
                                               -H     9AA                  65553.206930  
                     3-Hydroxyanthranilic acid +Cl    9AA                     41.606110  
                                               -H     9AA                   5885.244730  
                                               [M]-   9AA                   2547.082945

In [13]:
# Aggregate data per dataset and axis values
# Calculate what fraction metabolites in this dataset were detected with a given X, Y axis value
# There are 172 metaboites in total

n_metabolites = df.name_short.nunique()
step2 = step1.groupby(['dataset_id', X, Y]).agg({
                                                'effective_intensity':'mean',
                                                'detectability': lambda x: sum(x)/n_metabolites}) 


step2.head()

effective_intensity  detectability
dataset_id           adduct Matrix short                                    
2021-06-18_10h37m54s +Cl    9AA                    752.467978       0.360465
                     -H     9AA                  98242.664881       0.779070
                     [M]-   9AA                   7726.594086       0.186047
2021-06-18_10h41m59s +Cl    CHCA                    80.200649       0.029070
                     -H     CHCA                 11340.135429       0.523256

In [16]:
# Finally, take the average of results of all datasets

step3 = step2.groupby([X, Y]).agg({
                                'effective_intensity' : 'mean',
                                'detectability' : 'mean'})

step3.rename(columns={'detectability':'fraction_detected'}, inplace=True)
step3['log10_intensity'] = np.log10(step3['effective_intensity']+1)
step3.head()

effective_intensity  fraction_detected  log10_intensity
adduct Matrix short                                                         
+Cl    9AA                    752.467978           0.360465         2.877065
       CHCA                    80.200649           0.029070         1.909560
       CMBT                    97.961136           0.081395         1.995465
       ClCCA                   15.033169           0.023256         1.205019
       DAN                    411.088459           0.215116         2.614990

## Example for when you have class in axes

- chem_class.main_chem_class are chemical classes
- chem_subclass.fine_class are chemical subclasses
- pathways.main_coarse_path is the main pathway
- pathway_subclass.fine_path is the pathway "subclass"

In [8]:
# Here I use an example of fine chemical class
# I calcuate the size of each class first

chem = chem_subclass[chem_subclass.coarse_class != 'Thermometers']
sizes = chem_subclass.fine_class.value_counts()
chem['class_size'] = [sizes[k] for k in chem.fine_class]

<ipython-input-8-69516090f4e1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chem['class_size'] = [sizes[k] for k in chem.fine_class]


In [9]:
# for coarse class I would use chem_class[chem_class.main_coarse_class != 'Thermometers']... etc.

In [10]:
# Then I merge it with predictions
data = data.merge(chem, on='name_short', how='right')

In [12]:
# First step is to  aggregate per metabolite, dataset and axes values

# Let's use matrix and class as example axes values
X='Matrix short'
Y='fine_class'

step1 = data.pivot_table(index=['dataset_id', 'name_short', X, Y],
            values=['effective_intensity', 'detectability', 'class_size'],
            aggfunc=
                {'effective_intensity':'sum',
                'detectability':'max',
                'class_size':'first'},
            fill_value=0)

step1.head()

class_size  \
dataset_id           name_short              Matrix short fine_class                                      
2021-06-21_12h29m16s 4-Hydroxyproline        9AA          Polar amino acids                          10   
                     Acetylcarnitine         9AA          Quarternary ammonium amines                 8   
                     Acetylcholine           9AA          Quarternary ammonium amines                 8   
                     Adenosine monophosphate 9AA          Nucleotides                                14   
                     Agmatine                9AA          Arginine derivatives (guanidines)           5   

                                                                                             detectability  \
dataset_id           name_short              Matrix short fine_class                                         
2021-06-21_12h29m16s 4-Hydroxyproline        9AA          Polar amino acids                           True   
                     Acetylcarnitine         9AA          Quarternary ammonium amines                 True   
                     Acetylcholine           9AA          Quarternary ammonium amines                 True   
                     Adenosine monophosphate 9AA          Nucleotides                                 True   
                     Agmatine                9AA          Arginine derivatives (guanidines)           True   

                                                                                             effective_intensity  
dataset_id           name_short              Matrix short fine_class                                              
2021-06-21_12h29m16s 4-Hydroxyproline        9AA          Polar amino acids                           216.683670  
                     Acetylcarnitine         9AA          Quarternary ammonium amines               76946.032896  
                     Acetylcholine           9AA          Quarternary ammonium amines              192764.770000  
                     Adenosine monophosphate 9AA          Nucleotides                                 703.271155  
                     Agmatine                9AA          Arginine derivatives (guanidines)         23416.664830

In [13]:
# Next, aggregare per dataset and axes values

step2 = step1.pivot_table(index=['dataset_id', X,Y],
                      values=['effective_intensity', 'detectability', 'class_size'],
                      aggfunc = {
                                'class_size':'first',
                                'effective_intensity':'mean', # only when considering only 'detected' data
                                'detectability':'sum'
                                },
                      fill_value=0)

step2['fraction_detected'] = step2.detectability / step2.class_size
step2.head()

class_size  \
dataset_id           Matrix short fine_class                                      
2021-06-21_12h29m16s 9AA          Acidic amino acids                          8   
                                  Arginine derivatives (guanidines)           5   
                                  Aromatic amino acids                        6   
                                  Carbohydrate amines                         2   
                                  Fatty acyl                                  5   

                                                                     detectability  \
dataset_id           Matrix short fine_class                                         
2021-06-21_12h29m16s 9AA          Acidic amino acids                             2   
                                  Arginine derivatives (guanidines)              4   
                                  Aromatic amino acids                           1   
                                  Carbohydrate amines                            1   
                                  Fatty acyl                                     2   

                                                                     effective_intensity  \
dataset_id           Matrix short fine_class                                               
2021-06-21_12h29m16s 9AA          Acidic amino acids                         2604.696161   
                                  Arginine derivatives (guanidines)          7403.207476   
                                  Aromatic amino acids                         54.460793   
                                  Carbohydrate amines                         413.129500   
                                  Fatty acyl                               434835.169685   

                                                                     fraction_detected  
dataset_id           Matrix short fine_class                                            
2021-06-21_12h29m16s 9AA          Acidic amino acids                          0.250000  
                                  Arginine derivatives (guanidines)           0.800000  
                                  Aromatic amino acids                        0.166667  
                                  Carbohydrate amines                         0.500000  
                                  Fatty acyl                                  0.400000

In [14]:
# Finally, take the average of results of all datasets

step3 = step2.groupby([X, Y]).agg({
                                'effective_intensity' : 'mean',
                                'fraction_detected' : 'mean'})

step3['log10_intensity'] = step3['effective_intensity'].apply(lambda x: np.log10(x+1))
step3.head()

effective_intensity  \
Matrix short fine_class                                               
9AA          Acidic amino acids                         2604.696161   
             Arginine derivatives (guanidines)          7403.207476   
             Aromatic amino acids                         54.460793   
             Carbohydrate amines                         413.129500   
             Fatty acyl                               434835.169685   

                                                fraction_detected  \
Matrix short fine_class                                             
9AA          Acidic amino acids                          0.250000   
             Arginine derivatives (guanidines)           0.800000   
             Aromatic amino acids                        0.166667   
             Carbohydrate amines                         0.500000   
             Fatty acyl                                  0.400000   

                                                log10_intensity  
Matrix short fine_class                                          
9AA          Acidic amino acids                        3.415924  
             Arginine derivatives (guanidines)         3.869479  
             Aromatic amino acids                      1.743986  
             Carbohydrate amines                       2.617136  
             Fatty acyl                                5.638326